# Método Simplex Tableau
### Projeto M210 – Otimização

Preencha os coeficientes abaixo (até 4 variáveis e 4 restrições)
e execute todas as células para obter o resultado.

In [ ]:
# Quantidade de variáveis e restrições
n_vars = 4
n_rest = 4

# Coeficientes da função objetivo (Max Z)
# Exemplo: Z = 3x1 + 5x2 + 2x3 + 4x4
c = [3, 5, 2, 4]

# Matriz das restrições (coeficientes do lado esquerdo)
# Cada linha = uma restrição
A = [
    [2, 1, 1, 0],
    [1, 2, 0, 1],
    [1, 1, 2, 3],
    [0, 2, 1, 1]
]

# Lado direito das restrições (<=)
b = [8, 8, 10, 6]

In [ ]:
import numpy as np
from prettytable import PrettyTable

def simplex_tableau(c, A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    m, n = A.shape
    tableau = np.zeros((m + 1, n + m + 1))

    tableau[:m, :n] = A
    tableau[:m, n:n+m] = np.eye(m)
    tableau[:m, -1] = b
    tableau[-1, :n] = -np.array(c)
    return tableau

def mostrar_tableau(tableau):
    t = PrettyTable()
    for row in tableau:
        t.add_row([f"{x:.2f}" for x in row])
    print(t)

def simplex(c, A, b):
    tableau = simplex_tableau(c, A, b)
    m, n = len(A), len(A[0])
    iteracao = 0

    while any(tableau[-1, :-1] < 0):
        iteracao += 1
        print(f"\nIteração {iteracao}:")
        mostrar_tableau(tableau)

        col_pivo = np.argmin(tableau[-1, :-1])
        if all(tableau[:-1, col_pivo] <= 0):
            print("Solução ilimitada!")
            return None

        linhas_possiveis = tableau[:-1, -1] / tableau[:-1, col_pivo]
        linhas_possiveis[tableau[:-1, col_pivo] <= 0] = np.inf
        lin_pivo = np.argmin(linhas_possiveis)

        tableau[lin_pivo, :] /= tableau[lin_pivo, col_pivo]
        for i in range(len(tableau)):
            if i != lin_pivo:
                tableau[i, :] -= tableau[i, col_pivo] * tableau[lin_pivo, :]

    print("\nSolução ótima encontrada:")
    mostrar_tableau(tableau)

    n_vars = len(c)
    x = np.zeros(n_vars)
    for i in range(n_vars):
        col = tableau[:-1, i]
        if sum(col == 1) == 1 and sum(col) == 1:
            lin = np.where(col == 1)[0][0]
            x[i] = tableau[lin, -1]
    print(f"\nVariáveis básicas: {x}")
    print(f"Valor ótimo Z = {tableau[-1, -1]:.2f}")
    return x, tableau[-1, -1]

x, z = simplex(c, A, b)